In [1]:
from classes.ConditionalGAN import CWGAN
from utils.callbacks import WandbImagesConditionalGAN, SaveGeneratorWeights
import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback
import os
import tensorflow as tf

wandb.login()


wandb: Currently logged in as: matteoferrante (use `wandb login --relogin` to force relogin)


True

In [2]:

config={"dataset":"mnist","type":"CW-GAN"}

wandb.init(project="TorVergataExperiment-Generative",config=config)

## DATA


(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


###
BS=256
#strategy = tf.distribute.MirroredStrategy()

#with strategy.scope():
g=CWGAN(latent_dim=100,target_shape=(28,28,1),d_steps=5)

wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


c:\users\matte\anaconda3\envs\ai_env\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [3]:
g.discriminator.summary()

Model: "discriminator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        500         ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1, 784)       39984       ['embedding[0][0]']              
                                                                                                  
 input_2 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                      

In [4]:
g.gan.build(input_shape=(None,100))
g.gan.summary()

Model: "gan"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 generator (Functional)         (None, 28, 28, 1)    587256      ['input_7[0][0]',                
                                                                  'input_6[0][0]']                
                                                                                                  
 discriminator (Functional)     (None, 1)            1229093     ['generator[0][0]',            

In [5]:

ts=len(x_train)//BS

x_train=np.expand_dims(x_train.astype("float32")/255.,-1)
x_test=np.expand_dims(x_test.astype("float32")/255.,-1)



train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_dataset=  train_dataset.shuffle(buffer_size=1024).batch(BS).repeat()


test_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_dataset=test_dataset.shuffle(1024).batch(BS)


In [6]:

os.makedirs("models/wgan_mnist",exist_ok=True)
model_check=SaveGeneratorWeights(filepath="models/wgan_mnist/generator_gan_mnist.h5")



callbacks=[
    WandbImagesConditionalGAN(target_shape=(28,28,1)),
    WandbCallback(),
    model_check
]



TypeError: __init__() got an unexpected keyword argument 'target_shape'